In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Model
from keras.layers import Conv2D, Conv1D, Input, Reshape, Dense, Flatten
from keras.layers import BatchNormalization, Activation, AveragePooling1D
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from keras.regularizers import l2
import pickle

/usr/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
Using TensorFlow backend.


In [2]:
path = './TwoClass/'

In [3]:
xtrain = np.load(path + 'train_features.npy')
ytrain = np.load(path + 'train_labels.npy')

xval = np.load(path + 'val_features.npy')
yval = np.load(path + 'val_labels.npy')

xtest = np.load(path + 'test_features.npy')
ytest = np.load(path + 'test_labels.npy')

In [4]:
#### Model ####
reg = l2(.1)

X = Input(batch_shape = (None, 60, 41, 2))

conv1 = Conv2D(128, (3, 41), strides = (1,41), padding = 'same', 
               kernel_initializer = 'he_normal', kernel_regularizer = reg)(X)
bn1 = BatchNormalization()(conv1)
act1 = Activation('relu')(bn1)

reshape = Reshape((60,128))(act1)

conv2 = Conv1D(256, 3, strides = 1, padding = 'same', 
               kernel_initializer = 'he_normal', kernel_regularizer = reg)(reshape)
bn2 = BatchNormalization()(conv2)
act2 = Activation('relu')(bn2)

pool = AveragePooling1D(pool_size = act2.get_shape().as_list()[1], 
                        strides=1, padding='valid')(act2)

flat = Flatten()(pool)
d = Dense(2, kernel_initializer='he_normal', kernel_regularizer = reg)(flat)
bn_d = BatchNormalization()(d)
Yhat = Activation('softmax')(bn_d)

model = Model(X, Yhat)

#### Training Set Up ####

weight_dict = {0: float(ytrain[ytrain[:,0]==1].shape[0])/ytrain[ytrain[:,0]==1].shape[0],
               1: float(ytrain[ytrain[:,0]==1].shape[0])/ytrain[ytrain[:,1]==1].shape[0]}

lr = 1e-3                                                                                                                                                     
dcy = 0.0
m = 0.9                                                                                                                                                                           
sgd = SGD(lr=lr, momentum=m, decay=dcy,  nesterov=True)

name = './history/model1_lr%.1ef_dcy%.2f_m%.2f' % (lr, dcy, m)
checkpointer = ModelCheckpoint(filepath = name + '_{epoch:03d}-{val_acc:.2f}.h5',
                               monitor = 'val_acc', verbose=0, save_best_only=True)

model.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics = ['accuracy'])




In [5]:
### Training Model ###
a = model.fit(xtrain, ytrain, batch_size=512, epochs=1, verbose=1, callbacks = [checkpointer], 
          validation_data= (xval, yval), class_weight=weight_dict)



Train on 513 samples, validate on 70 samples
Epoch 1/1
513/513 [==============================] - 5s - loss: 21.0752 - acc: 0.6082 - val_loss: 22.7885 - val_acc: 0.3714


In [6]:
### Saving History ###
with open('history.pkl', 'wb') as f:
    pickle.dump(a.history, f)

In [10]:
# Graphing History
with open('history.pkl', 'rb') as f:
    a = pickle.load(f)


fig = plt.figure(figsize=(9,6))
plt.plot(a['val_acc'], label = 'validation accuracy')
plt.plot(a['acc'], label = 'training accuracy')
plt.axis([0,200,.2, 1.1])
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.title('Training with L2 Regularization')
plt.legend(loc = 4, frameon = 0)

plt.show()

EOFError: 